<font color="orange" size=25> <br>Lesson 7 <br><br></font>
<font color="black" size=6>1. Fire activity monitoring and prediction<br><br></font>
<font color="black" size=6>2. How to measure the tree height? <br><br>
</font>

<font color="orange" size=6> 1. Fire activity monitoring and prediction<br><br></font>
<font color="orange" size=5>**Load python libraries**<br><br></font>

In [ ]:
import os 
import numpy.ma as ma 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib import patches as mpatches, colors 
from matplotlib.colors import ListedColormap 
import matplotlib as mpl 
import seaborn as sns 
import geopandas as gpd 
from shapely.geometry import mapping 
import rasterio as rio 
from rasterio.plot import show, plotting_extent

sns.set_theme(font_scale=1.5) 
sns.set_style("white") 

os.chdir(r'D:\M\地理大数据与云计算\实验数据\lesson8\Lesson8_data') 


<font color="orange" size=5>**Calculate dNBR using Landsat data**<br><br></font>
<font color="black" size=4>**Open Landsat pre-fire and post-fire data**</font>

In [ ]:
with rio.open( 'cold_springs_fire/landsat_pre_fire.tif' ) as landsat_pre_fire: 
            landsat_pre_fire_data = landsat_pre_fire.read(masked=True) 
            landsat_pre_fire_meta = landsat_pre_fire.profile
            landsat_pre_fire_extent = plotting_extent(landsat_pre_fire) 
with rio.open( 'cold_springs_fire/landsat_post_fire.tif' ) as landsat_post_fire: 
            landsat_post_fire_data = landsat_post_fire.read(masked=True) 
            landsat_post_fire_meta = landsat_post_fire.profile
            landsat_post_fire_extent = plotting_extent(landsat_post_fire) 
# Open fire boundary layer and reproject it to match the Landsat data 
fire_boundary_path = "cold_springs_fire/co_cold_springs_20160711_2200_dd83.shp" 
fire_boundary = gpd.read_file(fire_boundary_path) 
# If the CRS' are not the same be sure to reproject 
fire_bound_utmz13 = fire_boundary.to_crs(landsat_pre_fire_meta['crs']) 


<font color="black" size=4>**Calculate and plot pre-fire NBR**</font>

In [ ]:
landsat_pre_fire_nbr = (landsat_pre_fire_data[4]-landsat_pre_fire_data[6]) / (landsat_pre_fire_data[4]+landsat_pre_fire_data[6]) 

fig, ax = plt.subplots(figsize=(12, 6)) 
nbr = ax.imshow(landsat_pre_fire_nbr, cmap='PiYG', vmin=-1, vmax=1, extent=landsat_pre_fire_extent) 
fire_bound_utmz13.plot(ax=ax, color='None',edgecolor='black', linewidth=2) 
fig.colorbar(nbr) 
ax.set(title="Landsat derived Normalized Burn Ratio\n 23 July 2016 \n Pre Cold Springs Fire") 
ax.set_axis_off() 
plt.show() 

<font color="black" size=4>**Calculate and plot post-fire NBR**</font>

In [ ]:
landsat_post_fire_nbr = (landsat_post_fire_data[4]-landsat_post_fire_data[6]) /(landsat_post_fire_data[4]+landsat_post_fire_data[6]) 
fig, ax = plt.subplots(figsize=(12, 6)) 
nbr = ax.imshow(landsat_post_fire_nbr, cmap='PiYG',vmin=-1, vmax=1, extent=landsat_post_fire_extent) 
fire_bound_utmz13.plot(ax=ax, color='None',edgecolor='black', linewidth=2) 
fig.colorbar(nbr) 
ax.set(title="Landsat derived Normalized Burn Ratio\n 23 July 2016 \n Post Cold Springs Fire") 
ax.set_axis_off() 
plt.show() 

<font color="black" size=4>**Calculate and plot dNBR**</font>

In [ ]:
# Calculate dnbr 
#请在此处输入代码
 
dnbr_cat_names = ["Enhanced Regrowth", 
                                "Unburned", 
                                "Low Severity", 
		   "Moderate Severity",
		   "High Severity"] 
nbr_colors = ["g", 
	       "yellowgreen", 
	       "peachpuff", 
	       "coral", 
	       "maroon"] 
nbr_cmap = ListedColormap(nbr_colors) 
class_bins = [dnbr_landsat.min(), -0.1, 0.1, 0.27, 0.66, np.inf] 
dnbr_landsat_class = np.digitize(dnbr_landsat, class_bins) 


In [ ]:
# Plot the data with a custom legend 
fig, ax = plt.subplots(figsize=(10, 8)) 
im = ax.imshow(dnbr_landsat_class, cmap=nbr_cmap, 
                          extent=landsat_pre_fire_extent) 
fire_bound_utmz13.plot(ax=ax, color='None', edgecolor='black', linewidth=2) 
values = np.unique(dnbr_landsat_class.ravel()) 
nbr_colors = [im.cmap(im.norm(value)) for value in values] 
patches = [mpatches.Patch(color=nbr_colors[i], 
                  label="{l}".format(l=dnbr_cat_names[i])) 
                  for i in range(len(dnbr_cat_names))] 
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), 
                 loc=2, borderaxespad=0., prop={'size': 13}) 
ax.set_title("Landsat dNBR - Cold Spring Fire Site \n June 1, 2017 - July 24, 2017", 
                  fontsize=16)
# turn off ticks 
ax.set_axis_off() 
plt.show() 


<font color="black" size=4>**Create a colorbar legend**</font>

In [ ]:
# Grab raster unique values (classes) 
values = np.unique(dnbr_landsat_class).tolist() 

# Add another index value because for n categories you need n+1 values to create bins 
values = [0] + values

# Make a color map of fixed colors 
nbr_colors = ["g", "yellowgreen", "peachpuff", "coral", "maroon"] 
nbr_cmap = ListedColormap(nbr_colors) 

# But the goal is the identify the MIDDLE point of each bin to create a centered tick
bounds = [((a + b) / 2) for a, b in zip(values[:-1], values[1::1])] + [5.5] 

# Define normalization 
norm = colors.BoundaryNorm(bounds, nbr_cmap.N) 

# Plot the data 
fig, ax = plt.subplots(figsize=(10, 8)) 
im = ax.imshow(dnbr_landsat_class, cmap=nbr_cmap, 
                          extent=landsat_pre_fire_extent, norm=norm) 
fire_bound_utmz13.plot(ax=ax, color='None', edgecolor='black', linewidth=2) 
cbar = fig.colorbar(im, boundaries=bounds, fraction=.035) 
cbar.set_ticks([np.unique(dnbr_landsat_class)]) 
cbar.set_ticklabels(dnbr_cat_names) 
ax.set_title("Landsat dNBR - Cold Spring Fire Site \n June 22, 2017 - July 24, 2017", 
                   fontsize=16) 

# turn off ticks 
ax.set_axis_off() 
plt.show() 

<font color="black" size=4>**Calculate total area of burned area**</font>

In [ ]:
# To calculate area, multiply the number of pixels in each bin by image resolution 
# Result will be in total square meters 
landsat_pixel = landsat_pre_fire.res[0] * landsat_pre_fire.res[0] 
burned_landsat = (dnbr_landsat_class[dnbr_landsat_class == 5]).size 
#请在此处输入代码
print("Landsat Severe Burn Pixels:", burned_landsat) 

<font color="orange" size=6><br>2. How to measure the tree height?<br><br></font>
<font color="orange" size=5>**Load python libraries**</font>

In [ ]:
import os 
from osgeo import gdal
import rasterio as rio 
from rasterio.plot import show, plotting_extent 
import matplotlib.pyplot as plt 
import numpy as np 

plt.ion() 
from shapely.geometry import Polygon, mapping 
from rasterio.mask import mask
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch
import geopandas as gpd 
import pandas as pd

plt.rcParams['figure.figsize'] = (8, 8) 
# Prettier plotting with seaborn 
import seaborn as sns
sns.set_theme(font_scale=1.5) 
sns.set_style("white") 

os.chdir(r'D:\M\地理大数据与云计算\实验数据\lesson8\Lesson8_data') 

<font color="orange" size=5>**Open raster data**<br><br></font>
<font color="black" size=4>**Read DTM raster**</font>

In [ ]:
# open raster data 
lidar_dtm = rio.open(r'D:\M\地理大数据与云计算\实验数据\lesson8\Lesson8_data\tree_height\SJER_lidarDTM.tif') 
lidar_dtm_im = lidar_dtm.read(1, masked=True) 


fig, ax = plt.subplots(figsize = (10,6)) 
dtm_plot = ax.imshow(lidar_dtm_im,cmap='Greys')
fig.colorbar(dtm_plot, ax=ax) 
ax.set_title("Lidar Digital Terrain Model") 
ax.set_axis_off() 

<font color="black" size=4>**Read DTM raster**</font>

In [ ]:
# open raster data 
lidar_dsm = rio.open(r'D:\M\地理大数据与云计算\实验数据\lesson8\Lesson8_data\tree_height\SJER_lidarDSM.tif') 
#请在此处输入代码

fig, ax = plt.subplots(figsize = (10,6)) 
dsm_plot = ax.imshow(lidar_dsm_im,cmap='Greys') 
fig.colorbar(dsm_plot, ax=ax) 
ax.set_title("Lidar Digital Surface Model") 
ax.set_axis_off() 



<font color="orange" size=5>**Calculate CHM data**<br><br></font>
<font color="black" size=4>**Before you subtract the two rasters, be sure to check to see if they cover the same area**</font>

In [ ]:
# are the bounds the same? 
print("Is the spatial extent the same?", lidar_dtm.bounds == lidar_dsm.bounds)
# is the resolution the same ?
print("Is the resolution the same?", lidar_dtm.res == lidar_dsm.res) 


<font color="black" size=4>**Subtract the two rasters**</font>

In [ ]:
# calculate canopy height model 
#请在此处输入代码

# plot the data 
fig, ax = plt.subplots(figsize = (10,6)) 
chm_plot = ax.imshow(lidar_chm_im, cmap='viridis') 
fig.colorbar(chm_plot, ax=ax) 
ax.set_title("Lidar Canopy Height Model (CHM)") 
ax.set_axis_off()

<font color="black" size=4>**Distribution of CHM value**</font>

In [ ]:
fig, ax = plt.subplots(figsize = (10,8)) 
ax.hist(lidar_chm_im.ravel()) 
ax.set_title("Histogram of CHM Values")

In [ ]:
print('CHM minimum value: ', lidar_chm_im.min()) 
print('CHM max value: ', lidar_chm_im.max()) 

<font color="black" size=4>**Update the nodata value**</font>

In [ ]:
# fill the masked pixels with a set no data value
nodatavalue = -999.0 
lidar_chm_im_fi = np.ma.filled(lidar_chm_im, fill_value=nodatavalue) 
lidar_chm_im_fi.min(), nodatavalue 

In [ ]:
# update the metadata to ensure the nodata value is properly documented 
# create dictionary copy 
chm_meta = lidar_dtm.meta.copy() 
# update the nodata value to be an easier to use number 
chm_meta.update({'nodata': nodatavalue}) 

<font color="black" size=4>**Export CHM raster layer**</font>

In [ ]:
out_path = "tree_height/SJER_lidarCHM.tif" 
with rio.open(out_path, 'w', **chm_meta) as outf: 
       outf.write(lidar_chm_im_fi, 1) 


<font color="orange" size=5>**Classify and plot CHM raster**<br><br></font>
<font color="black" size=4>**Map Raster Values to New Values**</font>

In [ ]:
# Reassign all values that are classified as 0 to masked (no data value) 
# This will prevent pixels that == 0 from being rendered on a map in matplotlib 
class_bins = [lidar_chm_im.min(), 2, 7, 12, np.inf] 
#请在此处输入代码
lidar_chm_class_ma = np.ma.masked_where(lidar_chm_im_class == 0 , lidar_chm_im_class, copy=True) 
 
# Note that you have an extra class in the data (0) 
print(np.unique(lidar_chm_im_class)) 

In [ ]:
# Plot data using nicer colors 
colors = ['linen', 'lightgreen', 'darkgreen', 'maroon'] 
cmap = ListedColormap(colors) 

fig, ax = plt.subplots(figsize=(10, 10)) 
chm_plot = ax.imshow(lidar_chm_class_ma, 
                                     cmap=cmap) 
fig.colorbar(chm_plot, ax=ax) 
ax.set_title("Classified Canopy Height Model") 
ax.set_axis_off() 


<font color="black" size=4>**Add Custom Legend**</font>

In [ ]:
# Create a list of labels to use for your legend 
height_class_labels = ["Short trees", "Less short trees", "Medium trees","Tall trees"] 
# In this case a patch is a box draw on your legend 
# Below you create a unique path or box with a unique color - one for each of the labels above 
legend_patches = [Patch(color=icolor, label=label) 
                              for icolor, label in zip(colors, height_class_labels)] 
cmap = ListedColormap(colors) 
fig, ax = plt.subplots(figsize=(10, 10)) 
ax.imshow(lidar_chm_class_ma, cmap=cmap) 
ax.legend(handles=legend_patches, facecolor ="white", edgecolor = "white", 
                 bbox_to_anchor = (1.4,1)) # Place legend to the RIGHT of the map 
ax.set_axis_off()

<font color="orange" size=5>**Compare Lidar derived height to in situ measurements**<br><br></font>
<font color="black" size=4>**Read CHM raster data**</font>

In [ ]:
# Load & plot the data 
with rio.open('tree_height/SJER_lidarCHM.tif') as sjer_lidar_chm_src: 
       SJER_chm_data = sjer_lidar_chm_src.read(1, masked=True) 
       sjer_chm_meta = sjer_lidar_chm_src.profile
       sjer_chm_plt = plotting_extent(sjer_lidar_chm_src)
fig, ax = plt.subplots(figsize=(8, 8)) 
ax.hist(SJER_chm_data.ravel()) 
ax.set_title('Distribution of Pixel Values \n Lidar Canopy Height Model', fontsize=18) 
ax.set_xlabel("Lidar Estimated Tree Height (m)") 

# Turn off scientific notation 
ax.ticklabel_format(useOffset=False, style='plain') 


In [ ]:
# View summary statistics of canopy height model 
print('Mean:', SJER_chm_data.mean()) 
print('Max:', SJER_chm_data.max()) 
print('Min:', SJER_chm_data.min()) 


<font color="black" size=4>**Clean up data - remove 0’s**</font>

In [ ]:
# Set CHM values of 0 to NAN (no data or not a number) 
SJER_chm_data[SJER_chm_data == 0] = np.nan 

# Remove nans, flatten the data & plot historgram 
SJER_chm_data_no_na = SJER_chm_data[~np.isnan(SJER_chm_data)].ravel()
 
fig, ax = plt.subplots(figsize=(10, 10)) 
ax.hist(SJER_chm_data_no_na) 
ax.set_title('Distribution of Pixel Values \n Lidar Canopy Height Model', fontsize=18) 
ax.set_xlabel("Lidar Estimated Tree Height (m)") 
ax.set_ylabel("Total Pixels") 

ax.ticklabel_format(useOffset=False, style='plain') 

<font color="black" size=4>**Load in situ points and overlay on top of the CHM raster Data**</font>

In [ ]:
sjer_centroids_path = r'D:\M\地理大数据与云计算\实验数据\lesson8\Lesson8_data\tree_height\SJER_plot_centroids.shp' 
SJER_plots_points = gpd.read_file(sjer_centroids_path)

# Overlay Points on Top Of Your Raster Data
fig, ax = plt.subplots(figsize=(15, 15)) 
ax.imshow(SJER_chm_data, 
                  # Here you must set the spatial extent or else the data will not line up
                  extent=plotting_extent(sjer_lidar_chm_src), cmap='Greys') 
SJER_plots_points.plot(ax=ax, marker='s', markersize=45, color='purple') 
ax.set_title("San Joachin Field Site \n Locations Vegetation Plot Locations", 
                   fontsize=20) 
plt.show()
 


<font color="black" size=4>**Create a 20 m buffer around each in situ point**</font>

In [ ]:
# Create a buffered polygon layer from your plot location points 
SJER_plots_poly = SJER_plots_points.copy() 

# Buffer each point using a 20 meter circle radius and replace the point geometry with the new buffered geometry 
#请在此处输入代码

# Export the buffered point layer as a shapefile to use in zonal stats 
plot_buffer_path = 'tree_height/plot_buffer.shp' 
SJER_plots_poly.to_file(plot_buffer_path) 


<font color="black" size=4>**Extract CHM pixel values for each buffer**</font>

In [ ]:
# Extract zonal stats 
from rasterstats import zonal_stats, point_query
sjer_tree_heights = zonal_stats(plot_buffer_path, 
                                                      SJER_chm_data, 
			           affine=sjer_chm_meta['transform'], 
			           geojson_out=True, 
			           copy_properties=True, 
			           stats="count min mean max median") 

# Turn extracted data into a pandas geodataframe 
SJER_lidar_height_df = gpd.GeoDataFrame.from_features(sjer_tree_heights) 
SJER_lidar_height_df.head() 


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5)) 
ax.bar(SJER_lidar_height_df['Plot_ID'], 
           SJER_lidar_height_df['max']) 
ax.set(xlabel="Plot ID", ylabel="Max Height") 
plt.setp(ax.get_xticklabels(), rotation=45, horizontalalignment='right') 
plt.show() 

In [ ]:
# Extract zonal stats but retain the individual pixel values 
sjer_tree_heights_ras = zonal_stats(plot_buffer_path, SJER_chm_data, 
				    affine=sjer_chm_meta['transform'], 
				    geojson_out=True, raster_out=True, 
				    copy_properties=True, 
				    stats="count min mean max median") 
# Convert to geodataframe 
SJER_lidar_height_df_ras = gpd.GeoDataFrame.from_features( sjer_tree_heights_ras) 
# View subset of the dataframe 
SJER_lidar_height_df_ras[["Plot_ID", "count", "geometry", 
                                           "mini_raster_affine", "mini_raster_array"]].head() 


In [ ]:
# Plot a histogram of pixel values for each plot 
n_columns = 3 
n_rows = int(np.ceil(len(SJER_lidar_height_df) / n_columns)) 
fig, axs = plt.subplots(n_rows, n_columns, figsize=(5*n_columns, 5*n_rows), 
                                   sharex=True, sharey=True) 

for (zone, ix), ax in zip(SJER_lidar_height_df.iterrows(), axs.ravel()): 
     data = SJER_lidar_height_df_ras.iloc[zone]['mini_raster_array'] 
     null_value = SJER_lidar_height_df_ras.iloc[zone]['mini_raster_nodata'] 
     data_values = data.data[data.mask] 
     data_values = data_values[~np.isnan(data_values)] 

     ax.hist(data_values) 
     ax.set(title=SJER_lidar_height_df_ras.iloc[zone]['Plot_ID'], xlabel="Raster Values") 
     plt.tight_layout() 

<font color="black" size=4>**Open in situ measured tree height file**</font>

In [ ]:
# Import & view in situ (field measured) data 
path_insitu = r'D:\M\地理大数据与云计算\实验数据\lesson8\Lesson8_data\tree_height\D17_2013_SJER_vegStr.csv' 
SJER_insitu_all = pd.read_csv(path_insitu) 
# View columns in data 
SJER_insitu_all.columns 

<font color="black" size=4>**Select just the columns that you will need in your analysis**</font>

In [ ]:
SJER_insitu = SJER_insitu_all[["siteid", "sitename", "plotid", "stemheight", "scientificname"]] 
SJER_insitu.head() 

<font color="black" size=4>**Summarize tree height data using Pandas**</font>

In [ ]:
insitu_stem_ht = SJER_insitu.groupby('plotid')['stemheight'] .agg( ['mean', 'max'])
insitu_stem_ht.head() 


<font color="black" size=4>**Add the word insitu to each column header so it’s very clear which data columns are human measured**</font>

In [ ]:
['insitu_' + col for col in insitu_stem_ht.columns] 

# Add insitu to each column name to make your data more expressive 
insitu_stem_ht.columns = ['insitu_' + col for col in insitu_stem_ht.columns] 

# Reset the index (plotid) 
insitu_stem_ht = insitu_stem_ht.reset_index() 
insitu_stem_ht.head() 

<font color="black" size=4>**Merge in situ data with spatial data.frame**</font>

In [ ]:
# Rename columns so that we know which columns represent lidar values 
SJER_lidar_height_df = SJER_lidar_height_df.rename( 
         columns={'max': 'lidar_max', 'mean': 'lidar_mean', 'min':'lidar_min'}) 
# Join lidar and human measured tree height data 
SJER_final_height = SJER_lidar_height_df.merge(insitu_stem_ht, 
                               left_on='Plot_ID', right_on='plotid') 

# Convert to a dataframe so you can use standard pandas plotting 
SJER_final_height_df = pd.DataFrame(SJER_final_height) 

# Export the final data frame as a csv file. 
SJER_final_height_df.to_csv( "tree_height/sjer-lidar-insitu-merge.csv") 


<font color="black" size=4>**Create map of in situ points sized by tree height**</font>

In [ ]:
# Convert the geometry column to contain points 
SJER_final_height['geometry'] = SJER_final_height.centroid 
SJER_final_height.head() 

SJER_final_height['insitu_max'] 

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 
ax.imshow(SJER_chm_data, cmap='Greys', extent=sjer_chm_plt) 
# Plot centroids of each geometry as points so that you can control their size 
SJER_final_height.centroid.plot(ax=ax, marker='o',  markersize=SJER_final_height['insitu_max'] * 80)
plt.show() 


<font color="orange" size=5>**Quantifying Relationships Between Variables**<br><br></font>
<font color="black" size=4>**Plot data (CHM vs Measured)**</font>

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) 
SJER_final_height_df.plot('lidar_max', 'insitu_max', 
                                          kind='scatter', fontsize=14, s=60, ax=ax) 
ax.set(xlabel="Lidar derived max tree height", ylabel="Measured tree height (m)") 
ax.set_title("Lidar vs Measured Max Tree Height \n NEON SJER Field Site", 
                  fontsize=30) 
plt.show() 


<font color="black" size=4>**Add a 1:1 line and making the x and y axis the same**</font>

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) 
SJER_final_height_df.plot('lidar_max', 'insitu_max', 
                                          kind='scatter', fontsize=14, s=60, ax=ax) 

ax.set(xlabel="Lidar Derived Max Tree Height", 
           ylabel="Measured Tree Height (m)") 
ax.set_title("Lidar vs Measured Tree Height - SJER", fontsize=30) 

# Add 1:1 line 
ax.plot((0, 1), (0, 1), transform=ax.transAxes, ls='--', c='k') 

# Adjust x and y axis limits 
ax.set(xlim=[0, 30], ylim=[0, 30]) 
plt.show() 


<font color="black" size=4>**Linear regression**</font>

In [ ]:
from scipy import stats 

x = SJER_final_height_df.lidar_max 
y = SJER_final_height_df.insitu_max 

#请在此处输入代码

print("slope:", slope, "\nintercept:", intercept, "\nr squared:", r_value**2) 


<font color="black" size=4>**Plot regression fits compared to 1:1**</font>

In [ ]:
# this plot should be a scatter plot with labels and such 
fig, ax = plt.subplots(figsize=(10, 10)) 
csfont = {'fontname':'Myriad Pro'} 
SJER_final_height_df.plot('lidar_max', 'insitu_max', 
                                          kind='scatter', fontsize=14, s=60, ax=ax, label = "Data") 

# Add a diagonal line 
ax.set(xlim=[0, 30], ylim=[0, 30]) 
x = x.values
ax.plot((0, 1), (0, 1), 'y-', transform=ax.transAxes, label = "1:1 line") 
ax.plot(x, intercept + slope*x, 'grey', label='regression fitted line') 

ax.set(xlabel="Lidar derived max tree height", ylabel="Measured tree height (m)") 
ax.set_title("Lidar vs Measured Tree Height - SJER", fontsize=30, **csfont) 
plt.legend()
plt.show()


<font color="black" size=4>**Use Seaborn to plot regression fit**</font>

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) 

csfont = {'fontname':'Myriad Pro'} 
ax = sns.regplot(x='lidar_max', y='insitu_max', data=SJER_final_height_df) 
ax.set_title("Lidar vs Measured Tree Height - SJER", fontsize=24) 

# Add a diagonal line 
ax.set(xlim=[5, 30], ylim=[5, 30]) 
ax.plot((0, 1), (0, 1), transform=ax.transAxes, ls='--', c='k') 

ax.set(xlabel="Lidar derived max tree height",ylabel="Measured tree height (m)")
plt.show() 
